<a href="https://colab.research.google.com/github/Ddhawan2003/FplAnalytics/blob/main/KnapsackFPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import requests

# Step 1: Load the existing CSV file
csv_file_path = 'updated_players.csv'
updated_players_df = pd.read_csv(csv_file_path)

# Step 2: Fetch the player prices from the Fantasy Premier League API
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    players = data['elements']

    # Create a dictionary to map player names to their now_cost
    player_prices = {
        f"{player['first_name']} {player['second_name']}": player['now_cost'] / 10.0  # Convert to millions
        for player in players
    }

    # Update the prices in the DataFrame
    updated_players_df['now_cost'] = updated_players_df['first_name'] + ' ' + updated_players_df['second_name']
    updated_players_df['now_cost'] = updated_players_df['now_cost'].map(player_prices).fillna(updated_players_df['now_cost'])

    # Step 3: Save the updated DataFrame back to CSV
    updated_players_df.to_csv(csv_file_path, index=False)
    print("Updated player prices successfully.")
else:
    print(f"Failed to retrieve data: {response.status_code}")


if 'price' in updated_players_df.columns:
    updated_players_df.drop(columns=['now_cost'], inplace=True)


Updated player prices successfully.


In [10]:
import pandas as pd

# Load the player data from the CSV file
players_df = pd.read_csv('cleaned_players.csv', delimiter=',')  # Change delimiter if necessary

def calculate_predicted_points(row):
    points = 0

    # Minutes played points
    if row['minutes'] >= 60:
        points += 2
    elif row['minutes'] > 0:
        points += 1

    # Goals scored points
    if row['element_type'] == 'GK':
        points += row['goals_scored'] * 10  # Goalkeeper goals
        points += row['clean_sheets'] * 4  # Clean sheets for GK
        points += row['goals_conceded'] // 2 * -1  # Goals conceded penalties
        points += row['bps'] // 3  # Bonus points from BPS
    elif row['element_type'] == 'DEF':
        points += row['goals_scored'] * 6  # Defender goals
        points += row['clean_sheets'] * 4  # Clean sheets for DEF
        points += row['goals_conceded'] // 2 * -1  # Goals conceded penalties
        points += row['bps'] // 3  # Bonus points from BPS
    elif row['element_type'] == 'MID':
        points += row['goals_scored'] * 5  # Midfielder goals
        points += row['assists'] * 3  # Assists points for MID
        points += row['clean_sheets'] * 1  # Clean sheets for MID
        points += row['bps'] // 3  # Bonus points from BPS
    elif row['element_type'] == 'FWD':
        points += row['goals_scored'] * 4  # Forward goals
        points += row['assists'] * 3  # Assists points for FWD
        points += row['bps'] // 3  # Bonus points from BPS

    # Penalties, cards, and own goals penalties
    points += row['yellow_cards'] * -1
    points += row['red_cards'] * -3

    return points

# Apply the function to each row to calculate predicted points
players_df['predicted_points'] = players_df.apply(calculate_predicted_points, axis=1)

# Save the updated DataFrame back to CSV
players_df.to_csv('updated_players.csv', index=False)

print("Predicted points have been calculated and saved to 'updated_players.csv'.")


Predicted points have been calculated and saved to 'updated_players.csv'.


In [27]:
import pandas as pd
import pulp

# Load the updated_players.csv file
csv_file_path = 'updated_players.csv'
updated_players_df = pd.read_csv(csv_file_path)

# Convert now_cost to price (the cost is in millions, so we divide by 10)
updated_players_df['price'] = updated_players_df['now_cost']

# Create a list of players with relevant information
players = []
for index, row in updated_players_df.iterrows():
    players.append({
        'name': f"{row['first_name']} {row['second_name']}",
        'points': row['predicted_points'],
        'price': row['price'],
        'position': row['element_type'],  # Assuming element_type corresponds to position
    })

# Initialize the problem
problem = pulp.LpProblem("Best_Premier_League_Team", pulp.LpMaximize)

# Create decision variables
x = pulp.LpVariable.dicts("player", range(len(players)), cat="Binary")

# Objective function: maximize total predicted points
problem += pulp.lpSum([x[i] * players[i]['points'] for i in range(len(players))])

# Budget constraint
problem += pulp.lpSum([x[i] * players[i]['price'] for i in range(len(players))]) <= 100

# Position constraints
problem += pulp.lpSum([x[i] for i in range(len(players)) if players[i]['position'] == 'GK']) == 2
problem += pulp.lpSum([x[i] for i in range(len(players)) if players[i]['position'] == 'DF']) <= 5
problem += pulp.lpSum([x[i] for i in range(len(players)) if players[i]['position'] == 'MF']) <= 5
problem += pulp.lpSum([x[i] for i in range(len(players)) if players[i]['position'] == 'FW']) <= 3

# Solve the problem
problem.solve()

# Get the selected team
selected_team = [players[i]['name'] for i in range(len(players)) if x[i].value() == 1]
total_points = pulp.value(problem.objective)

# Output the results
print(f"Selected team: {selected_team}")
print(f"Total predicted points: {total_points}")


Selected team: ['Bryan Mbeumo', 'Danny Welbeck', 'Noni Madueke', 'Cole Palmer', 'Dwight McNeil', 'Emile Smith Rowe', 'Wilfred Ndidi', 'Alisson Ramses Becker', 'Ibrahima Konaté', 'Luis Díaz', 'Virgil van Dijk', 'Mateo Kovačić', 'Diogo Dalot Teixeira', 'André Onana', 'James Maddison', 'Cristian Romero']
Total predicted points: 1217.0
